# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd 
import requests 
import xlsxwriter 
import math
from scipy import stats 
import configparser

In [2]:
# Loading keys from config parser
config = configparser.ConfigParser()
config.read_file(open('/home/venom/GitHub/cheatsheets/courses/freecodecamp/algorithmic_trading_with_python/secret.cfg'))

In [3]:
IEX_SANDBOX_PRIVATE = config.get('IEX', 'IEX_SANDBOX_PRIVATE')
IEX_SANDBOX_PUBLIC = config.get('IEX', 'IEX_SANDBOX_PUBLIC')

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [4]:
stocks = pd.read_csv('/home/venom/GitHub/cheatsheets/courses/freecodecamp/algorithmic_trading_with_python/data/sp_500_stocks.csv')

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [5]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_SANDBOX_PUBLIC}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2245281187550,
 'week52high': 143.3,
 'week52low': 58.03,
 'week52change': 0.7007907951631017,
 'sharesOutstanding': 16923162784,
 'float': 0,
 'avg10Volume': 99147071,
 'avg30Volume': 108132404,
 'day200MovingAvg': 117.71,
 'day50MovingAvg': 129.03,
 'employees': 0,
 'ttmEPS': 3.44,
 'ttmDividendRate': 0.8093097141284469,
 'dividendYield': 0.00618179752407058,
 'nextDividendDate': '0',
 'exDividendDate': '2020-11-02',
 'nextEarningsDate': '0',
 'peRatio': 39.16416929926773,
 'beta': 1.2161086713810885,
 'maxChangePercent': 49.72876519813577,
 'year5ChangePercent': 5.094998656526241,
 'year2ChangePercent': 2.469590885902867,
 'year1ChangePercent': 0.6957673947094603,
 'ytdChangePercent': -0.005112076854344871,
 'month6ChangePercent': 0.3481051618041233,
 'month3ChangePercent': 0.1285178763744531,
 'month1ChangePercent': 0.04383431187330028,
 'day30ChangePercent': 0.02990687424977209,
 'day5ChangePercent': 0.008760515442055245}

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [6]:
data['year1ChangePercent']

0.6957673947094603

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [7]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [8]:
final_dataframe = pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_SANDBOX_PUBLIC}'
    
    data = requests.get(batch_api_call_url).json()
    
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['quote']['latestPrice'],
                    data[symbol]['stats']['year1ChangePercent'],
                    'N/A'
                ],
                index=my_columns
            ),
            ignore_index=True
        )

In [9]:
final_dataframe.head()

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,128.370,0.440715,N/A
1,AAL,15.779,-0.437528,N/A
2,AAP,171.050,0.104548,N/A
3,AAPL,139.430,0.676634,N/A
4,ABBV,114.540,0.365031,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [10]:
final_dataframe.sort_values('One-Year Price Return', ascending=False, inplace=True)
final_dataframe = final_dataframe[:51]
final_dataframe.reset_index(drop=True, inplace=True)
final_dataframe.head()

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,CARR,40.759,2.3463,N/A
1,FCX,31.596,1.47043,N/A
2,ALB,181.100,1.31626,N/A
3,LB,47.097,1.31342,N/A
4,NVDA,551.400,1.18263,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [11]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

portfolio_input()
print(portfolio_size)

Enter the value of your portfolio: 10000000


10000000


In [12]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe['Price'][i])
final_dataframe.head()

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,CARR,40.759,2.3463,4810
1,FCX,31.596,1.47043,6205
2,ALB,181.100,1.31626,1082
3,LB,47.097,1.31342,4163
4,NVDA,551.400,1.18263,355


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [13]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'
]

hqm_dataframe = pd.DataFrame(columns=hqm_columns)

for symbol_string in symbol_strings:
    
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_SANDBOX_PUBLIC}'
    data = requests.get(batch_api_call_url).json()
    
    for symbol in symbol_string.split(','):
        
        hqm_dataframe = hqm_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['quote']['latestPrice'],
                    'N/A',
                    data[symbol]['stats']['year1ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['month6ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['month3ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['month1ChangePercent'],
                    'N/A',
                    'N/A'
                ],
                index=hqm_columns
            ),
            ignore_index=True
        )

In [14]:
hqm_dataframe.columns

Index(['Ticker', 'Price', 'Number of Shares to Buy', 'One-Year Price Return',
       'One-Year Return Percentile', 'Six-Month Price Return',
       'Six-Month Return Percentile', 'Three-Month Price Return',
       'Three-Month Return Percentile', 'One-Month Price Return',
       'One-Month Return Percentile', 'HQM Score'],
      dtype='object')

In [15]:
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,131.570,N/A,0.454457,N/A,0.351825,N/A,0.222142,N/A,0.0780707,N/A,N/A
1,AAL,16.277,N/A,-0.444959,N/A,0.416589,N/A,0.26882,N/A,-0.0206344,N/A,N/A
2,AAP,169.740,N/A,0.105192,N/A,0.146737,N/A,0.0824226,N/A,0.0325666,N/A,N/A
3,AAPL,137.120,N/A,0.679707,N/A,0.3558,N/A,0.128695,N/A,0.0445416,N/A,N/A
4,ABBV,116.300,N/A,0.349977,N/A,0.162358,N/A,0.35873,N/A,0.0953666,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,113.860,N/A,0.0581207,N/A,0.220975,N/A,0.130997,N/A,-0.00342304,N/A,N/A
501,ZBH,165.640,N/A,0.107965,N/A,0.261261,N/A,0.158957,N/A,0.0789349,N/A,N/A
502,ZBRA,420.480,N/A,0.611579,N/A,0.479855,N/A,0.392409,N/A,0.0905183,N/A,N/A
503,ZION,48.930,N/A,-0.0124759,N/A,0.557459,N/A,0.622125,N/A,0.181108,N/A,N/A


In [16]:
hqm_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 505 entries, 0 to 504
Data columns (total 12 columns):
Ticker                           505 non-null object
Price                            505 non-null float64
Number of Shares to Buy          505 non-null object
One-Year Price Return            501 non-null object
One-Year Return Percentile       505 non-null object
Six-Month Price Return           501 non-null object
Six-Month Return Percentile      505 non-null object
Three-Month Price Return         501 non-null object
Three-Month Return Percentile    505 non-null object
One-Month Price Return           501 non-null object
One-Month Return Percentile      505 non-null object
HQM Score                        505 non-null object
dtypes: float64(1), object(11)
memory usage: 47.5+ KB


In [17]:
convert_dict = {
    # 'Number of Shares to Buy': float,
    'One-Year Price Return': float,
    # 'One-Year Return Percentile': float,
    'Six-Month Price Return': float,
    # 'Six-Month Return Percentile': float,
    'Three-Month Price Return': float,
    # 'Three-Month Return Percentile': float,
    'One-Month Price Return': float,
    # 'One-Month Return Percentile': float,
    # 'HQM Score': float,
}

In [18]:
hqm_dataframe = hqm_dataframe.astype(convert_dict)

## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [19]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])/100

# Print each percentile score to make sure it was calculated properly
for time_period in time_periods:
    print(hqm_dataframe[f'{time_period} Return Percentile'])

#Print the entire DataFrame    
hqm_dataframe

/home/venom/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:1942: RuntimeWarning: invalid value encountered in less
  left = np.count_nonzero(a < score)
/home/venom/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:1943: RuntimeWarning: invalid value encountered in less_equal
  right = np.count_nonzero(a <= score)


0       0.879208
1      0.0158416
2       0.526733
3       0.940594
4       0.813861
         ...    
500     0.433663
501     0.528713
502     0.920792
503     0.326733
504     0.611881
Name: One-Year Return Percentile, Length: 505, dtype: object
0      0.651485
1      0.752475
2      0.336634
3      0.663366
4      0.356436
         ...   
500    0.477228
501    0.534653
502    0.807921
503    0.859406
504    0.287129
Name: Six-Month Return Percentile, Length: 505, dtype: object
0      0.637624
1       0.70495
2      0.334653
3      0.457426
4      0.786139
         ...   
500    0.461386
501    0.524752
502    0.815842
503    0.948515
504    0.231683
Name: Three-Month Return Percentile, Length: 505, dtype: object
0      0.677228
1      0.166337
2      0.467327
3      0.546535
4      0.738614
         ...   
500    0.253465
501    0.681188
502    0.720792
503    0.922772
504    0.263366
Name: One-Month Return Percentile, Length: 505, dtype: object


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,131.570,N/A,0.454457,0.879208,0.351825,0.651485,0.222142,0.637624,0.078071,0.677228,N/A
1,AAL,16.277,N/A,-0.444959,0.0158416,0.416589,0.752475,0.268820,0.70495,-0.020634,0.166337,N/A
2,AAP,169.740,N/A,0.105192,0.526733,0.146737,0.336634,0.082423,0.334653,0.032567,0.467327,N/A
3,AAPL,137.120,N/A,0.679707,0.940594,0.355800,0.663366,0.128695,0.457426,0.044542,0.546535,N/A
4,ABBV,116.300,N/A,0.349977,0.813861,0.162358,0.356436,0.358730,0.786139,0.095367,0.738614,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,113.860,N/A,0.058121,0.433663,0.220975,0.477228,0.130997,0.461386,-0.003423,0.253465,N/A
501,ZBH,165.640,N/A,0.107965,0.528713,0.261261,0.534653,0.158957,0.524752,0.078935,0.681188,N/A
502,ZBRA,420.480,N/A,0.611579,0.920792,0.479855,0.807921,0.392409,0.815842,0.090518,0.720792,N/A
503,ZION,48.930,N/A,-0.012476,0.326733,0.557459,0.859406,0.622125,0.948515,0.181108,0.922772,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [20]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

In [21]:
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,131.570,N/A,0.454457,0.879208,0.351825,0.651485,0.222142,0.637624,0.078071,0.677228,0.711386
1,AAL,16.277,N/A,-0.444959,0.0158416,0.416589,0.752475,0.268820,0.70495,-0.020634,0.166337,0.409901
2,AAP,169.740,N/A,0.105192,0.526733,0.146737,0.336634,0.082423,0.334653,0.032567,0.467327,0.416337
3,AAPL,137.120,N/A,0.679707,0.940594,0.355800,0.663366,0.128695,0.457426,0.044542,0.546535,0.65198
4,ABBV,116.300,N/A,0.349977,0.813861,0.162358,0.356436,0.358730,0.786139,0.095367,0.738614,0.673762
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,113.860,N/A,0.058121,0.433663,0.220975,0.477228,0.130997,0.461386,-0.003423,0.253465,0.406436
501,ZBH,165.640,N/A,0.107965,0.528713,0.261261,0.534653,0.158957,0.524752,0.078935,0.681188,0.567327
502,ZBRA,420.480,N/A,0.611579,0.920792,0.479855,0.807921,0.392409,0.815842,0.090518,0.720792,0.816337
503,ZION,48.930,N/A,-0.012476,0.326733,0.557459,0.859406,0.622125,0.948515,0.181108,0.922772,0.764356


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [22]:
hqm_dataframe.sort_values(by='HQM Score', ascending=False, inplace=True)
hqm_dataframe = hqm_dataframe[:51]
hqm_dataframe.reset_index(drop=True, inplace=True)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,ALB,181.480,N/A,1.275445,0.986139,1.134606,0.976238,0.963843,0.980198,0.310786,0.984158,0.981683
1,FCX,31.165,N/A,1.469333,0.990099,1.332551,0.988119,0.766143,0.972277,0.261878,0.974257,0.981188
2,SIVB,467.650,N/A,0.808539,0.968317,1.170305,0.978218,0.692861,0.962376,0.297066,0.982178,0.972772
3,GM,57.334,N/A,0.596620,0.916832,1.179369,0.982178,0.589385,0.936634,0.379337,0.992079,0.956931
4,AMAT,109.600,N/A,0.758762,0.954455,0.725853,0.922772,0.740157,0.970297,0.252726,0.972277,0.95495
5,LB,47.800,N/A,1.295536,0.988119,1.561192,0.990099,0.389658,0.807921,0.180001,0.918812,0.926238
6,LRCX,582.390,N/A,0.875797,0.970297,0.609018,0.885149,0.570220,0.924752,0.170702,0.910891,0.922772
7,APTV,147.100,N/A,0.614422,0.924752,0.802889,0.942574,0.462651,0.871287,0.169416,0.906931,0.911386
8,AES,28.473,N/A,0.416197,0.861386,0.879807,0.958416,0.431500,0.857426,0.244656,0.964356,0.910396
9,MU,85.065,N/A,0.467555,0.885149,0.643479,0.90099,0.570916,0.926733,0.172427,0.912871,0.906436


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [23]:
portfolio_input()

Enter the value of your portfolio: 10000000


In [24]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)
for i in range(0, len(hqm_dataframe['Ticker'])):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe['Price'][i])
hqm_dataframe

/home/venom/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,ALB,181.480,1080,1.275445,0.986139,1.134606,0.976238,0.963843,0.980198,0.310786,0.984158,0.981683
1,FCX,31.165,6291,1.469333,0.990099,1.332551,0.988119,0.766143,0.972277,0.261878,0.974257,0.981188
2,SIVB,467.650,419,0.808539,0.968317,1.170305,0.978218,0.692861,0.962376,0.297066,0.982178,0.972772
3,GM,57.334,3419,0.596620,0.916832,1.179369,0.982178,0.589385,0.936634,0.379337,0.992079,0.956931
4,AMAT,109.600,1789,0.758762,0.954455,0.725853,0.922772,0.740157,0.970297,0.252726,0.972277,0.95495
5,LB,47.800,4102,1.295536,0.988119,1.561192,0.990099,0.389658,0.807921,0.180001,0.918812,0.926238
6,LRCX,582.390,336,0.875797,0.970297,0.609018,0.885149,0.570220,0.924752,0.170702,0.910891,0.922772
7,APTV,147.100,1332,0.614422,0.924752,0.802889,0.942574,0.462651,0.871287,0.169416,0.906931,0.911386
8,AES,28.473,6886,0.416197,0.861386,0.879807,0.958416,0.431500,0.857426,0.244656,0.964356,0.910396
9,MU,85.065,2305,0.467555,0.885149,0.643479,0.90099,0.570916,0.926733,0.172427,0.912871,0.906436


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [25]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [26]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [27]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', integer_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [28]:
writer.save()